In [1]:
from PyDBDdim.utils import DimensionlessLearning
from vics_fcns import top_split_y#, top_split_x
import torch
torch.manual_seed(0)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")


#First, we must load our data.

In [2]:
#df = pd.ExcelFile(r"mass_balance_params.xlsx")
#df_2 = pd.ExcelFile(r'C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\He_code_new\collected_output_files_packing\data_from_EB_looping_all_sizes_remove_extraneous_dims.xlsx')
df_2 = pd.ExcelFile(r'C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\data\data_no_packing_all\data_from_EB_looping_new_excel_new_code_no_packing_v3.xlsx')

In [3]:
# df_train_x['u_B_m_s'] = df_train_x['u_B_m_s'] / 1000 if np.min(df_train_x['u_B_m_s']) > 0 else df_train_x['u_B_m_s'] / 1000 - (np.min(df_train_x['u_B_m_s']) / 1000-1e-5)
# df_train_x['A_tot_m2'] = 1.
# df_train_x['t_a_s'] = df_train_x['t_a_s'] / 1e-7 if np.min(df_train_x['t_a_s']) > 0 else df_train_x['t_a_s'] / 1e-7 - (np.min(df_train_x['t_a_s']) / 1e-7-1e-5)
# df_train_x['t_b_s'] = df_train_x['t_b_s'] / 1e-4 if np.min(df_train_x['t_b_s']) > 0 else df_train_x['t_b_s'] / 1e-4 - (np.min(df_train_x['t_b_s']) / 1e-4-1e-5)
# df_train_x['Volume_m3'] = 1.
# df_train_x['K_iz_a_m3_s_atom'] = df_train_x['K_iz_a_m3_s_atom'] / 1e-15 if np.min(df_train_x['K_iz_a_m3_s_atom']) > 0 else df_train_x['K_iz_a_m3_s_atom'] / 1e-15 - (np.min(df_train_x['K_iz_a_m3_s_atom']) / 1e-15-1e-5)

# df_train_x['K_2_iz_a_m3_s_atom'] = df_train_x['K_2_iz_a_m3_s_atom'] / 1e-15 if np.min(df_train_x['K_2_iz_a_m3_s_atom']) > 0 else df_train_x['K_2_iz_a_m3_s_atom'] / 1e-15 - (np.min(df_train_x['K_2_iz_a_m3_s_atom']) / 1e-15-1e-5)

# df_train_x['K_iz_exc_a_m3_s_atom'] = df_train_x['K_iz_exc_a_m3_s_atom'] / 1e-14 if np.min(df_train_x['K_iz_exc_a_m3_s_atom']) > 0 else df_train_x['K_iz_exc_a_m3_s_atom'] / 1e-14 - (np.min(df_train_x['K_iz_exc_a_m3_s_atom']) / 1e-14-1e-5)

# df_train_x['n_sa_atoms_m3'] = df_train_x['n_sa_atoms_m3'] / 1e10 if np.min(df_train_x['n_sa_atoms_m3']) > 0 else df_train_x['n_sa_atoms_m3'] / 1e10 - (np.min(df_train_x['n_sa_atoms_m3']) / 1e10-1e-5)

# df_train_x['n_sb_atoms_m3'] = df_train_x['n_sb_atoms_m3'] / 1e10 if np.min(df_train_x['n_sb_atoms_m3']) > 0 else df_train_x['n_sb_atoms_m3'] / 1e10 - (np.min(df_train_x['n_sb_atoms_m3']) / 1e10-1e-5)

# df_train_x['A_a_m2'] = 1.

# df_train_x['A_b_m2'] = 1.

# df_train_x['n_He_exc_a_atoms_m3'] = df_train_x['n_He_exc_a_atoms_m3'] / 1e17 if np.min(df_train_x['n_He_exc_a_atoms_m3']) > 0 else df_train_x['n_He_exc_a_atoms_m3'] / 1e17 - (np.min(df_train_x['n_He_exc_a_atoms_m3']) / 1e17-1e-5)

def rescale(g):
    for i in range(0,g.shape[1]):
        if np.min(np.abs(g[:,i])) ==0:
            n = 0
        else:
            n = np.mean((np.log10(np.min(np.abs(g[:,i]))), np.log10(np.max(np.abs(g[:,i])))))
        if n<0:
                g[:,i] = g[:,i]/10**np.ceil(n) if np.min(g[:,i]) > 0 else g[:,i]/10**np.ceil(n) - (np.min(g[:,i]) / 10**np.ceil(n)-1e-5)
        else:
            g[:,i] = g[:,i]/10**np.floor(n) if np.min(g[:,i]) > 0 else g[:,i]/10**np.floor(n) - (np.min(g[:,i]) / 10**np.floor(n)-1e-5)
    return g

def rescale_vec(g):
    if np.min(np.abs(g)) ==0:
        n = 0
    else:
        n = np.mean((np.log10(np.min(np.abs(g))), np.log10(np.max(np.abs(g)))))
    if n<0:
            g= g/10**np.ceil(n) if np.min(g) > 0 else g/10**np.ceil(n) - (np.min(g) / 10**np.ceil(n)-1e-5)
    else:
        g = g/10**np.floor(n) if np.min(g) > 0 else g/10**np.floor(n) - (np.min(g) / 10**np.floor(n)-1e-5)
    return g

In [4]:
#                   0             1           2                    3                   4                   5           6                7                    8                     9                        10                  11                  12                      13                          
#df1_names = [ 't_a_seconds', 'Q_a_As', 'V_p_ta_kgm2_s3_A','T_e_a_kgm2_s3_A', 'n_He_exc_a_atoms_m3', 'u_B_a_m_s', 'v_e_a_m_s', 'K_2_iz_a_m3_s_atom','K_loss_a_m6_s_atom2', 'K_iz_exc_a_m3_s_atom', 'K_exc_a_m3_s_atom', 'K_iz_a_m3_s_atom', 'K_elastic_a_m3_s_atom', 'E_elastic_a_kgm2_s2' ]


df_time_a = rescale(np.array(pd.read_excel(df_2, sheet_name='time_a_data').iloc[:,1:]))
df_time_a_units = np.array(pd.read_excel(df_2, sheet_name='time_a_data_units').iloc[:,1:])
df_time_a_n = pd.read_excel(df_2, sheet_name='time_a_data_names').iloc[:,1:]
time_a_n= [df_time_a_n.iloc[0,i] for i in range(0,df_time_a_n.shape[1])]
#print(df_time_a_n)
#print(df_time_a_units)


#                   0             1                2               3                   4              5          6             7                   8                    9                       10              11                   12                   13                  
#df2_names =  [ 't_b_seconds', 'Q_b_As', 'V_p_tb_kgm2_s3_A','T_e_kgm2_s3_A', 'n_He_exc_atoms_m3', 'u_B_m_s', 'v_e_m_s','K_2_iz_m3_s_atom','K_loss_m6_s_atom2', 'K_iz_exc_m3_s_atom', 'K_exc_m3_s_atom', 'K_iz_m3_s_atom', 'K_elastic_m3_s_atom', 'E_elastic_kgm2_s2' ]
df_time_b = rescale(np.array(pd.read_excel(df_2, sheet_name='time_b_data').iloc[:,1:]))
df_time_b_units = np.array(pd.read_excel(df_2, sheet_name='time_b_data_units').iloc[:,1:])
df_time_b_n = pd.read_excel(df_2, sheet_name='time_b_data_names').iloc[:,1:]
time_b_n= [df_time_b_n.iloc[0,i] for i in range(0,df_time_b_n.shape[1])]


#                    0                   1                 2               3                 4               5               6                 7                  8           9        10        11                12                   13               14                   
#df3_names = ['E_period_kgm2_s2', 'n_sa_atoms_m3','n_sb_atoms_m3', 'n_e_electrons_m3', 'n_g_atoms_m3', 'T_g_kelvin', 'E_iz_kgm2_s2', 'E_iz_exc_kgm2_s2', 'E_exc_kgm2_s2', 'e_c_As', 'm_e_kg', 'M_He_kg',  'epsilon_A2s4_kg_m3', 'eps_0_A2s4_kg_m3', 'k_b_kgm2_s2_K']
df_other = rescale(np.array(pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]))
df_other_units = np.array(pd.read_excel(df_2, sheet_name='other_data_units').iloc[:,1:])
df_other_n = pd.read_excel(df_2, sheet_name='other_data_names').iloc[:,1:]
other_n= [df_other_n.iloc[0,i] for i in range(0,df_other_n.shape[1])]

#                    0                 1             2        3        4         5            6            7            8               9              10                 11                      12                      
#df4_names = ['Volume_rxtor_m2', 'V_all_beads_m2','A_a_m2','A_b_m2', 'h_m', 'Volume_m3', 'A_bead_m2', 'A_tot_m3', 'frequency_Hz', 'Flow_m3_s', 'temp_C_gas_K', 'Set_Voltage_kgm2_s3_A', 'pulse_time_seconds' ]
df_exp = rescale(np.array(pd.read_excel(df_2, sheet_name='Experiment_Design_data').iloc[:,1:]))
df_exp_units = np.array(pd.read_excel(df_2, sheet_name='Experiment_Data_units').iloc[:,1:])
df_exp_n = pd.read_excel(df_2, sheet_name='Experiment_Data_names').iloc[:,1:]
exp_n= [df_exp_n.iloc[0,i] for i in range(0,df_exp_n.shape[1])]




# These next cells are about assembling process variables for dimmensionless numbers. Uncomment the cell you wish to run.

# You must prepare inputs (which contain the rescaled data), D_in (the dimensions matrix), and for ease of trackign I recommend your produce variable names (variables).

In [5]:
# # all terms
# a = df_time_b[:,5].shape[0]
# inputs = np.hstack([df_time_a,df_time_b, df_other, df_exp])
# D_in = np.hstack([df_time_a_units, df_time_b_units, df_other_units, df_exp_units])
# variables = time_a_n + time_b_n + other_n + exp_n
# print(D_in.shape)
# print(len(variables))
# print(variables[31]) #delete #31 for the MB.
# inputs = np.delete(inputs, 31, axis = 1)
# D_in = np.delete(D_in, 31, axis = 1)
# variables.pop(31)
# print(variables[31]) #good check!

In [6]:
# all MB terms
# time a:
a = df_time_b[:,5].shape[0]
# u_B_a [5], K_iz_a [11], K_iz_exc_a [9], n_He_exc_a [4], K_2_iz_a [7], t_a [0]
ta_inputs = np.hstack((df_time_a[:,5].reshape(a,1), df_time_a[:,11].reshape(a,1), df_time_a[:,9].reshape(a,1), df_time_a[:,4].reshape(a,1), df_time_a[:,7].reshape(a,1), df_time_a[:,0].reshape(a,1)))
ta_D_in = np.hstack((df_time_a_units[:,5].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_time_a_units[:,9].reshape(6,1), df_time_a_units[:,4].reshape(6,1), df_time_a_units[:,7].reshape(6,1), df_time_a_units[:,0].reshape(6,1)))
ta_n = [time_a_n[5], time_a_n[11], time_a_n[9], time_a_n[4], time_a_n[7], time_a_n[0] ]


# time b
# u_B [5], K_iz [11], n_He_exc [4], K_iz_exc [9], t_b [0], K_2_iz [7]       
tb_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,11].reshape(a,1), df_time_b[:,4].reshape(a,1), df_time_b[:,9].reshape(a,1), df_time_b[:,0].reshape(a,1), df_time_b[:,7].reshape(a,1)))
tb_D_in = np.hstack((df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,11].reshape(6,1), df_time_b_units[:,4].reshape(6,1), df_time_b_units[:,9].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_time_b_units[:,7].reshape(6,1)))
tb_n = [time_b_n[5], time_b_n[11], time_b_n[4], time_b_n[9], time_b_n[0], time_b_n[7] ]


# df other
# ng [4] , n_sa [1], n_sb [2], 
other_inputs = np.hstack(( df_other[:,4].reshape(a,1), df_other[:,1].reshape(a,1), df_other[:,2].reshape(a,1) ))
other_D_in = np.hstack(( df_other_units[:,4].reshape(6,1), df_other_units[:,1].reshape(6,1), df_other_units[:,2].reshape(6,1)  ))
other_n_in = [ other_n[4], other_n[1], other_n[2] ]
print(other_n_in)
# print(len(other_n_in))
print(other_D_in.shape)
print(other_inputs.shape)

#df experimental
# A_tot [7], Volume [5], A_a [2]
exp_inputs = np.hstack(( df_exp[:,7].reshape(a,1), df_exp[:,5].reshape(a,1), df_exp[:,2].reshape(a,1)  ))
exp_D_in = np.hstack(( df_exp_units[:,7].reshape(6,1), df_exp_units[:,5].reshape(6,1), df_exp_units[:,2].reshape(6,1) ))
exp_n_in = [ exp_n[7], exp_n[5], exp_n[2] ]

#all together
inputs = np.hstack(( ta_inputs, tb_inputs, other_inputs, exp_inputs))
D_in = np.hstack(( ta_D_in, tb_D_in, other_D_in, exp_D_in ))
variables = ta_n+tb_n+other_n_in+exp_n_in
print(variables)
print(len(variables))
# A_tot, Volume, A_a


#try again without n_He_exc

['n_g_atoms_m3', 'n_sa_atoms_m3', 'n_sb_atoms_m3']
(6, 3)
(48, 3)
['u_B_a_m_s', 'K_iz_a_m3_s_atom', 'K_iz_exc_a_m3_s_atom', 'n_He_exc_a_atoms_m3', 'K_2_iz_a_m3_s_atom', 't_a_seconds', 'u_B_m_s', 'K_iz_m3_s_atom', 'n_He_exc_atoms_m3', 'K_iz_exc_m3_s_atom', 't_b_seconds', 'K_2_iz_m3_s_atom', 'n_g_atoms_m3', 'n_sa_atoms_m3', 'n_sb_atoms_m3', 'A_tot_m3', 'Volume_m3', 'A_a_m2']
18


In [7]:
# # two major rates  same for packed / unpacked
# # include iomization ta, wall loss tb

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]



# inputs = np.hstack((ion_inputs, wall_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in ))
# variables = ion_n+wall_n
# print(variables)
# print(D_in)

# #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [8]:
# # two major rates + n_He_exca_rate + u_B_ms - n_He_exc_a
# # include iomization ta, wall loss tb, and he exc ion ta

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]

# # ta He double ioniz : V [na], n_He_exc_a  ta[4], K_2_iz_a  ta[7], t_a [na],
# he_inputs = np.hstack(( df_time_a[:,5].reshape(a,1), df_time_a[:,7].reshape(a,1) ))
# he_D_in = np.hstack(( df_time_a_units[:,5].reshape(6,1), df_time_a_units[:,7].reshape(6,1) ))
# he_n = [time_a_n[5], time_a_n[7] ]


# inputs = np.hstack((ion_inputs, wall_inputs, he_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in, he_D_in ))
# variables = ion_n+wall_n+he_n
# print(variables)
# print(D_in)

# #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [9]:
# # # three major rates  same for packed / unpacked
# # # include iomization ta, wall loss tb, tb loss to electrode a

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]

# # tb loss to electrode a : n_sa df_other[1] ,    A_a df_exp[2]  ,   u_B   df_time_b[na], t_b [na]
# elec_a_tb_inputs = np.hstack(( df_other[:,1].reshape(a,1) , df_exp[:,2].reshape(a,1) ))
# elec_a_tb_D_in = np.hstack(( df_other_units[:,1].reshape(6,1) , df_exp_units[:,2].reshape(6,1) ))
# elec_a_tb_n = [ other_n[1], exp_n[2]]

# inputs = np.hstack((ion_inputs, wall_inputs, elec_a_tb_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in, elec_a_tb_D_in ))
# variables = ion_n+wall_n + elec_a_tb_n
# print(variables)
# print(D_in)

# # #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [10]:
# # # four major rates, same for packed / unpacked
# # # include iomization ta, wall loss tb, tb loss to electrode a, tb loss electrode b

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]

# # tb loss to electrode a : n_sa df_other[1] ,    A_a df_exp[2]  ,   u_B   df_time_b[na], t_b [na]
# elec_a_tb_inputs = np.hstack(( df_other[:,1].reshape(a,1) , df_exp[:,2].reshape(a,1) ))
# elec_a_tb_D_in = np.hstack(( df_other_units[:,1].reshape(6,1) , df_exp_units[:,2].reshape(6,1) ))
# elec_a_tb_n = [ other_n[1], exp_n[2]]

# #tb loss to electrode b: n_sb df_other[2], A_a, [na], u_B [na], t_b [na]
# elec_b_tb_inputs =  df_other[:,2].reshape(a,1)
# elec_b_tb_D_in = df_other_units[:,2].reshape(6,1) 
# elec_b_tb_n = [other_n[2]]



# inputs = np.hstack((ion_inputs, wall_inputs, elec_a_tb_inputs, elec_b_tb_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in, elec_a_tb_D_in , elec_b_tb_D_in))
# variables = ion_n+wall_n + elec_a_tb_n + elec_b_tb_n
# print(variables)
# print(D_in)

# # #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [11]:
# # # five major rates--packed
# # # include iomization ta, wall loss tb, tb loss to electrode a, tb loss electrode b
# #  and ta packing loss

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]

# # tb loss to electrode a : n_sa df_other[1] ,    A_a df_exp[2]  ,   u_B   df_time_b[na], t_b [na]
# elec_a_tb_inputs = np.hstack(( df_other[:,1].reshape(a,1) , df_exp[:,2].reshape(a,1) ))
# elec_a_tb_D_in = np.hstack(( df_other_units[:,1].reshape(6,1) , df_exp_units[:,2].reshape(6,1) ))
# elec_a_tb_n = [ other_n[1], exp_n[2]]

# #tb loss to electrode b: n_sb df_other[2], A_a, [na], u_B [na], t_b [na]
# elec_b_tb_inputs =  df_other[:,2].reshape(a,1)
# elec_b_tb_D_in = df_other_units[:,2].reshape(6,1) 
# elec_b_tb_n = [other_n[2]]

# # ta packing loss: n_e [na], u_B_a df_time_a[:,5], A_tot [na], ta [na]
# packing_ta_inputs =  df_time_a[:,5].reshape(a,1)
# packing_ta_D_in = df_time_a_units[:,5].reshape(6,1) 
# packing_ta_n = [time_a_n[5]]


# inputs = np.hstack((ion_inputs, wall_inputs, elec_a_tb_inputs, elec_b_tb_inputs, packing_ta_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in, elec_a_tb_D_in , elec_b_tb_D_in, packing_ta_D_in))
# variables = ion_n+wall_n + elec_a_tb_n + elec_b_tb_n+packing_ta_n
# print(variables)
# print(D_in)

# # #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [12]:
# # # five major rates--unpacked
# # # include iomization ta, wall loss tb, tb loss to electrode a, tb loss electrode b
# #  and ta loss electrode a

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]

# # tb loss to electrode a : n_sa df_other[1] ,    A_a df_exp[2]  ,   u_B   df_time_b[na], t_b [na]
# elec_a_tb_inputs = np.hstack(( df_other[:,1].reshape(a,1) , df_exp[:,2].reshape(a,1) ))
# elec_a_tb_D_in = np.hstack(( df_other_units[:,1].reshape(6,1) , df_exp_units[:,2].reshape(6,1) ))
# elec_a_tb_n = [ other_n[1], exp_n[2]]

# #tb loss to electrode b: n_sb df_other[2], A_a, [na], u_B [na], t_b [na]
# elec_b_tb_inputs =  df_other[:,2].reshape(a,1)
# elec_b_tb_D_in = df_other_units[:,2].reshape(6,1) 
# elec_b_tb_n = [other_n[2]]

# # ta loss electrode a: n_sa[na] * u_B_a df_time_a[:,5] * A_a[na] * t_a [na]
# ta_loss_elec_a_inputs = df_time_a[:,5].reshape(a,1)
# ta_loss_elec_a_D_in = df_time_a_units[:,5].reshape(6,1)
# ta_loss_elec_a_n = [ time_a_n[5] ]


# inputs = np.hstack((ion_inputs, wall_inputs, elec_a_tb_inputs, elec_b_tb_inputs, ta_loss_elec_a_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in, elec_a_tb_D_in , elec_b_tb_D_in, ta_loss_elec_a_D_in))
# variables = ion_n+wall_n + elec_a_tb_n + elec_b_tb_n+ ta_loss_elec_a_n 
# print(variables)
# print(D_in)

# # #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [13]:
# # # six major rates--packed case
# # # include iomization ta, wall loss tb, tb loss to electrode a, tb loss electrode b
# #  # ta packing loss, ta Vol double He exc ionization

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]

# # tb loss to electrode a : n_sa df_other[1] ,    A_a df_exp[2]  ,   u_B   df_time_b[na], t_b [na]
# elec_a_tb_inputs = np.hstack(( df_other[:,1].reshape(a,1) , df_exp[:,2].reshape(a,1) ))
# elec_a_tb_D_in = np.hstack(( df_other_units[:,1].reshape(6,1) , df_exp_units[:,2].reshape(6,1) ))
# elec_a_tb_n = [ other_n[1], exp_n[2]]

# #tb loss to electrode b: n_sb df_other[2], A_a, [na], u_B [na], t_b [na]
# elec_b_tb_inputs =  df_other[:,2].reshape(a,1)
# elec_b_tb_D_in = df_other_units[:,2].reshape(6,1) 
# elec_b_tb_n = [other_n[2]]

# # ta packing loss: n_e [na], u_B_a df_time_a[:,5], A_tot [na], ta [na]
# packing_ta_inputs =  df_time_a[:,5].reshape(a,1)
# packing_ta_D_in = df_time_a_units[:,5].reshape(6,1) 
# packing_ta_n = [time_a_n[5]]

# # ta vol double He exc ionization: V exp[na], n_He_exc_a df_time_a[:,4]  ,  K_2_iz_a  df_time_a[:,7], ta [na]
# double_He_exc_ion_ta_inputs =  np.hstack(( df_time_a[:,4].reshape(a,1) , df_time_a[:,7].reshape(a,1) ))
# double_He_exc_ion_ta_D_in = np.hstack(( df_time_a_units[:,4].reshape(6,1), df_time_a_units[:,7].reshape(6,1) ))
# double_He_exc_ion_ta_n = [time_a_n[4], time_a_n[7]]


# inputs = np.hstack((ion_inputs, wall_inputs, elec_a_tb_inputs, elec_b_tb_inputs, packing_ta_inputs, double_He_exc_ion_ta_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in, elec_a_tb_D_in , elec_b_tb_D_in, packing_ta_D_in, double_He_exc_ion_ta_D_in))
# variables = ion_n+wall_n + elec_a_tb_n + elec_b_tb_n+packing_ta_n + double_He_exc_ion_ta_n
# print(variables)
# print(D_in)

# # #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [14]:
# # # NO n_he_exc_a --- six major rates--packed case--- NO n_he_exc_a
# # # include iomization ta, wall loss tb, tb loss to electrode a, tb loss electrode b
# #  # ta packing loss, ta Vol double He exc ionization

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]

# # tb loss to electrode a : n_sa df_other[1] ,    A_a df_exp[2]  ,   u_B   df_time_b[na], t_b [na]
# elec_a_tb_inputs = np.hstack(( df_other[:,1].reshape(a,1) , df_exp[:,2].reshape(a,1) ))
# elec_a_tb_D_in = np.hstack(( df_other_units[:,1].reshape(6,1) , df_exp_units[:,2].reshape(6,1) ))
# elec_a_tb_n = [ other_n[1], exp_n[2]]

# #tb loss to electrode b: n_sb df_other[2], A_a, [na], u_B [na], t_b [na]
# elec_b_tb_inputs =  df_other[:,2].reshape(a,1)
# elec_b_tb_D_in = df_other_units[:,2].reshape(6,1) 
# elec_b_tb_n = [other_n[2]]

# # ta packing loss: n_e [na], u_B_a df_time_a[:,5], A_tot [na], ta [na]
# packing_ta_inputs =  df_time_a[:,5].reshape(a,1)
# packing_ta_D_in = df_time_a_units[:,5].reshape(6,1) 
# packing_ta_n = [time_a_n[5]]

# # ta vol double He exc ionization: V exp[na], n_He_exc_a df_time_a[:,4]  ,  K_2_iz_a  df_time_a[:,7], ta [na]
# double_He_exc_ion_ta_inputs = df_time_a[:,7].reshape(a,1) 
# double_He_exc_ion_ta_D_in =  df_time_a_units[:,7].reshape(6,1) 
# double_He_exc_ion_ta_n = [time_a_n[7]]


# inputs = np.hstack((ion_inputs, wall_inputs, elec_a_tb_inputs, elec_b_tb_inputs, packing_ta_inputs, double_He_exc_ion_ta_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in, elec_a_tb_D_in , elec_b_tb_D_in, packing_ta_D_in, double_He_exc_ion_ta_D_in))
# variables = ion_n+wall_n + elec_a_tb_n + elec_b_tb_n+packing_ta_n + double_He_exc_ion_ta_n
# print(variables)
# print(D_in)

# # #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [15]:
# # six and seven major rates--unpacked. The sixth major rate, ta packing wall loss, has no new terms and
# # hence provides no new information. 
# # include iomization ta, wall loss tb, tb loss to electrode a, tb loss electrode b
# #  ta loss electrode a, ta packing wall loss, ta_vol_double_He_exc_ion

# vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
a = df_time_b[:,5].shape[0]
ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]

# tb loss to electrode a : n_sa df_other[1] ,    A_a df_exp[2]  ,   u_B   df_time_b[na], t_b [na]
elec_a_tb_inputs = np.hstack(( df_other[:,1].reshape(a,1) , df_exp[:,2].reshape(a,1) ))
elec_a_tb_D_in = np.hstack(( df_other_units[:,1].reshape(6,1) , df_exp_units[:,2].reshape(6,1) ))
elec_a_tb_n = [ other_n[1], exp_n[2]]

#tb loss to electrode b: n_sb df_other[2], A_a, [na], u_B [na], t_b [na]
elec_b_tb_inputs =  df_other[:,2].reshape(a,1)
elec_b_tb_D_in = df_other_units[:,2].reshape(6,1) 
elec_b_tb_n = [other_n[2]]

# ta loss electrode a: n_sa[na] * u_B_a df_time_a[:,5] * A_a[na] * t_a [na]
ta_loss_elec_a_inputs = df_time_a[:,5].reshape(a,1)
ta_loss_elec_a_D_in = df_time_a_units[:,5].reshape(6,1)
ta_loss_elec_a_n = [ time_a_n[5] ]

# ta packing wall loss: n_e [na], u_B_a [na], A_tot [na], t_a [na]
# # all redundant

# # ta vol double He exc ionization: V exp[na], n_He_exc_a df_time_a[:,4]  ,  K_2_iz_a  df_time_a[:,7], ta [na]
double_He_exc_ion_ta_inputs =  np.hstack(( df_time_a[:,4].reshape(a,1) , df_time_a[:,7].reshape(a,1) ))
double_He_exc_ion_ta_D_in = np.hstack(( df_time_a_units[:,4].reshape(6,1), df_time_a_units[:,7].reshape(6,1) ))
double_He_exc_ion_ta_n = [time_a_n[4], time_a_n[7]]

inputs = np.hstack((ion_inputs, wall_inputs, elec_a_tb_inputs, elec_b_tb_inputs, ta_loss_elec_a_inputs, double_He_exc_ion_ta_inputs))
D_in = np.hstack(( ion_D_in, wall_D_in, elec_a_tb_D_in , elec_b_tb_D_in, ta_loss_elec_a_D_in, double_He_exc_ion_ta_D_in))
variables = ion_n+wall_n + elec_a_tb_n + elec_b_tb_n+ ta_loss_elec_a_n +double_He_exc_ion_ta_n
print(variables)
print(D_in)



['t_a_seconds', 'K_iz_a_m3_s_atom', 'n_g_atoms_m3', 'Volume_m3', 'u_B_m_s', 't_b_seconds', 'A_tot_m3', 'n_sa_atoms_m3', 'A_a_m2', 'n_sb_atoms_m3', 'u_B_a_m_s', 'n_He_exc_a_atoms_m3', 'K_2_iz_a_m3_s_atom']
[[ 0  3 -3  3  1  0  2 -3  2 -3  1 -3  3]
 [ 1 -1  0  0 -1  1  0  0  0  0 -1  0 -1]
 [ 0 -1  1  0  0  0  0  1  0  1  0  1 -1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [16]:
# # two major rates + just the variable n_He_exc_a
# # include iomization ta, wall loss tb

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7], n_He_exc_a  ta[4]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) , df_time_a[:,4].reshape(a,1)))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1), df_time_a_units[:,4].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7], time_a_n[4] ]



# inputs = np.hstack((ion_inputs, wall_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in ))
# variables = ion_n+wall_n
# print(variables)
# print(D_in)

# #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [17]:
# # two major rates + n_He_exca_rate 
# # include iomization ta, wall loss tb, and he exc ion ta

# # vol time a ionization: t_a ta[0],, K_iz_a   ta[11], n_e [na], n_g other[4], V exp[5]
# a = df_time_b[:,5].shape[0]
# ion_inputs = np.hstack((df_time_a[:,0].reshape(a,1), df_time_a[:,11].reshape(a,1), df_other[:,4].reshape(a,1), df_exp[:,5].reshape(a,1) ))
# ion_D_in = np.hstack(( df_time_a_units[:,0].reshape(6,1), df_time_a_units[:,11].reshape(6,1), df_other_units[:,4].reshape(6,1), df_exp_units[:,5].reshape(6,1) ))
# ion_n = [time_a_n[0], time_a_n[11], other_n[4], exp_n[5] ]

# # vol time b packing wall loss: n_e [na], u_B tb[5], t_b  tb[0], A_tot exp[7]
# wall_inputs = np.hstack((df_time_b[:,5].reshape(a,1), df_time_b[:,0].reshape(a,1), df_exp[:,7].reshape(a,1) ))
# wall_D_in = np.hstack(( df_time_b_units[:,5].reshape(6,1), df_time_b_units[:,0].reshape(6,1), df_exp_units[:,7].reshape(6,1) ))
# wall_n = [time_b_n[5], time_b_n[0], exp_n[7] ]

# # ta He double ioniz : V [na], n_He_exc_a  ta[4], K_2_iz_a  ta[7], t_a [na],
# he_inputs = np.hstack(( df_time_a[:,4].reshape(a,1), df_time_a[:,7].reshape(a,1) ))
# he_D_in = np.hstack(( df_time_a_units[:,4].reshape(6,1), df_time_a_units[:,7].reshape(6,1) ))
# he_n = [time_a_n[4], time_a_n[7] ]


# inputs = np.hstack((ion_inputs, wall_inputs, he_inputs))
# D_in = np.hstack(( ion_D_in, wall_D_in, he_D_in ))
# variables = ion_n+wall_n+he_n
# print(variables)
# print(D_in)

# #A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,

In [18]:
# base terms and will add h and uB
#base terms + uba,  'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca', 'uBa', 'h'
#a = df_time_b[:,5].shape[0]
#inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1),df_time_a[:,5].reshape(a,1),df_exp[:,4].reshape(a,1)))
#D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1),df_time_a_units[:,5].reshape(6,1),df_exp_units[:,4].reshape(6,1)))
#variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4],time_a_n[5],exp_n[4]]
#print(variables)
#print(D_in)

# base terms with uB, minus nHeexca
#base terms + uba,  'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca', 'uBa'
# a = df_time_b[:,5].shape[0]
# inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,5].reshape(a,1)))
# D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,5].reshape(6,1)))
# variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[5]]
# print(variables)
# print(D_in)


# base terms and will add uB
#base terms + uba,  'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca', 'uBa'
# a = df_time_b[:,5].shape[0]
# inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1),df_time_a[:,5].reshape(a,1)))
# D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1),df_time_a_units[:,5].reshape(6,1)))
# variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4],time_a_n[5]]
# print(variables)
# print(D_in)


#base terms + F +K_iz, base terms: 'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,
#F is exp_n[9], K_iz is time_b_n[11]
#a = df_time_b[:,5].shape[0]
#inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1),df_exp[:,9].reshape(a,1),df_time_b[:,11].reshape(a,1)))
#D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1),df_exp_units[:,9].reshape(6,1),df_time_b_units[:,11].reshape(6,1)))
#variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4],exp_n[9], time_b_n[11]]
#print(variables)
#print(D_in)


#base terms + F, base terms: 'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,
#F is exp_n[9]
#a = df_time_b[:,5].shape[0]
#inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1),df_exp[:,9].reshape(a,1)))
#D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1),df_exp_units[:,9].reshape(6,1)))
#variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4],exp_n[9]]
#print(variables)
#print(D_in)


# base terms  'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,
# a = df_time_b[:,5].shape[0]
# inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1)))
# D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1)))
# variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4]]
# print(variables)
# print(D_in)

#base terms -A,V 'uB',  'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb',  'nHeexca,
# a = df_time_b[:,5].shape[0]
# inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_exp[:,2].reshape(a,1),df_time_a[:,4].reshape(a,1)))
# D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_time_a_units[:,4].reshape(6,1)))
# variables = [time_b_n[5],time_a_n[0],time_b_n[0],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],time_a_n[4]]
# print(variables)
# print(D_in)

#to compare to ketong's original code only
#not to construct  'uB', 'Atot', 'ta', 'tb', 'Volume', 'Ka', 'K2a', 'Kexca', 'nsa', 'nsb', 'Aa', 'Ab', 'nHeexca,
# a = df_time_b[:,5].shape[0] #this is just the number of data points
# inputs = np.hstack((df_time_b[:,5].reshape(a,1),df_exp[:,7].reshape(a,1),df_time_a[:,0].reshape(a,1),df_time_b[:,0].reshape(a,1),df_exp[:,5].reshape(a,1),df_time_a[:,11].reshape(a,1),df_time_a[:,7].reshape(a,1),df_time_a[:,9].reshape(a,1),df_other[:,1].reshape(a,1),df_other[:,2].reshape(a,1),df_exp[:,2].reshape(a,1),df_exp[:,3].reshape(a,1),df_time_a[:,4].reshape(a,1)))
# D_in = np.hstack((df_time_b_units[:,5].reshape(6,1),df_exp_units[:,7].reshape(6,1),df_time_a_units[:,0].reshape(6,1),df_time_b_units[:,0].reshape(6,1),df_exp_units[:,5].reshape(6,1),df_time_a_units[:,11].reshape(6,1),df_time_a_units[:,7].reshape(6,1),df_time_a_units[:,9].reshape(6,1),df_other_units[:,1].reshape(6,1),df_other_units[:,2].reshape(6,1),df_exp_units[:,2].reshape(6,1),df_exp_units[:,3].reshape(6,1),df_time_a_units[:,4].reshape(6,1)))
# variables = [time_b_n[5],exp_n[7],time_a_n[0],time_b_n[0],exp_n[5],time_a_n[11],time_a_n[7],time_a_n[9],other_n[1],other_n[2],exp_n[2],exp_n[3],time_a_n[4]]
# print(variables)
# print(D_in)


In [19]:
a

48

In [20]:
print(inputs.shape)
print(D_in.shape)

(48, 13)
(6, 13)


In [21]:
print(inputs[:,4])

[1.73383193 1.72345143 1.92042988 1.98369526 2.00907405 1.75139783
 1.71300804 1.92975108 2.00065023 2.17409766 1.78660233 1.8877228
 1.67247618 2.05581503 1.91216756 1.67562882 1.87763096 1.69815509
 1.7818696  1.92372487 1.84301989 1.69691147 1.6975334  1.726511
 1.72345143 1.73322303 1.9286568  2.0211218  2.00012256 2.157038
 2.08993244 1.79603038 1.93684878 1.93029798 1.87875495 1.99695363
 2.15262938 2.16046071 1.98209829 2.157038   1.90774635 1.87030853
 2.07268931 2.64062231 2.39495204 1.90719298 2.3201822  2.52120047]


# Next, we must produce our target variables, include their dimensions ve

In [22]:
#For predicting Te/Tg: keep top block, comment bottom block. For predicting ne/ng, keep bottom block, comment top block.




#### TOP BLOCK ####

# df_out = pd.read_excel(df_2, sheet_name='time_b_data').iloc[:,1:]
# T_e = np.array(df_out.iloc[:,3])
# df_out = pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]
# T_g = np.array(df_out.iloc[:,5])*0.026/297
# T_e_no_dim = T_e/T_g
# print(T_e)
# output = rescale_vec(T_e_no_dim)
# print(output)
# D_out = np.array(
#     [
#         [0.],
#         [0.],
#         [0.],
#     ]
# )




#### BOTTOM BLOCK ####

df_out = pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]
n_e = np.array(df_out.iloc[:,3])
df_out = pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]
n_g = np.array(df_out.iloc[:,4])
n_e_no_dim = n_e/n_g
print(n_e)
print(n_e_no_dim)
output = rescale_vec(n_e_no_dim)
print(output)
D_out = np.array(
    [
        [0.],
        [0.],
        [0.],
        [0.],
        [0,],
        [0.]
    ],

)


[3.88496592e+12 8.40777158e+12 1.84935323e+12 4.25936461e+12
 1.52310600e+12 4.80836688e+12 3.18989622e+12 3.88829674e+12
 3.41572870e+12 1.60334052e+12 2.27147985e+12 1.43269087e+12
 5.45898365e+12 1.89631062e+12 3.99972174e+12 8.99767792e+12
 2.62653575e+12 9.81651028e+12 3.21371282e+12 2.41836809e+12
 3.88826310e+12 3.00058897e+12 9.85800589e+12 6.53727255e+12
 1.60799513e+13 2.91132946e+13 1.53315588e+12 2.52187327e+12
 2.11925785e+12 1.18618153e+12 1.83666202e+12 3.74674399e+12
 1.24121146e+12 5.16922311e+11 1.81490079e+12 1.01293491e+12
 1.46015934e+12 8.21923413e+11 7.66117709e+11 9.93852477e+11
 1.83129375e+12 2.00980064e+12 2.11766942e+12 4.04949778e+11
 8.91117102e+11 1.40750655e+12 3.30500172e+11 7.00186721e+11]
[1.33695274e-13 2.89340845e-13 6.36427167e-14 1.46579643e-13
 5.24154078e-14 1.65472732e-13 1.09775493e-13 1.33809899e-13
 1.17547179e-13 5.51765584e-14 7.81695712e-14 4.93039068e-14
 1.87862732e-13 6.52586848e-14 1.37644423e-13 3.09641586e-13
 9.03882873e-14 3.37820

In [23]:
r = n_e/n_g
print(max(r))
print(min(r))
del r

1.0018903547745756e-12
1.1373667577272438e-14


# Now, we produce the nullspace basis vectors.

In [24]:
fff = DimensionlessLearning(D_in, D_out)
fff.basis_col

tensor([[ 1.0000,  1.0000, -1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  1.0000,
          1.0000,  0.0000],
        [ 1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,
          1.0000, -1.0000],
        [ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, -0.3333,  0.0000, -0.6667,  0.0000, -0.6667,  0.0000, -0.3333,
          0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  

In [16]:
fff.basis_col

tensor([[ 1.0000,  1.0000, -1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  1.0000,
          1.0000,  0.0000],
        [ 1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,
          1.0000, -1.0000],
        [ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, -0.3333,  0.0000, -0.6667,  0.0000, -0.6667,  0.0000, -0.3333,
          0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  

In [25]:
fff.read_data(inputs, output)
print(fff.X)

tensor([[ 0.1140,  0.8232,  2.9058,  ...,  0.2564,  0.6266,  1.3640],
        [ 0.1400,  0.6274,  2.9058,  ...,  0.2564,  0.5287,  1.3415],
        [ 0.1550,  0.5893,  2.9058,  ...,  0.2564,  0.3972,  1.3365],
        ...,
        [ 0.1650,  0.5585,  2.9058,  ...,  0.2564,  1.2821,  1.3322],
        [ 0.1650,  0.1736,  2.9058,  ...,  0.2564,  4.0607,  1.2470],
        [ 0.1700,  0.0746,  2.9058,  ...,  0.2564, 10.0573,  1.1936]],
       dtype=torch.float64)


# Now we prepare for dimensionless number discovery. 
# In this next cell, we choose how many different initial guess to start the optimization with, as well as where to store the results.
# summary path: where the overview of each result
good path: where dimensionless numbers with a r^2 > 0.3 are stored.
bad path: where dimensionless numbers with a r^2 < 0.3 are stored.

In [26]:
from PyDBDdim import PiLinearRegressionViaTorch
import pandas as pd

#the code does five fold to ensure the R2 is worth while.
# we then want to rerun the fitting on all data instead of 80% of it. The 5fold find the right number of epochs.
# so, we re run to find the right number of epochs, and then we must store the parameters and the R2 from the 5-fold fitting.
#
small_loop = [i for i in range(1,3)]
j =[i for i in range(1,6)]
h = [5*i for i in range(6,25)] # rerun for unpacked, 6/7 rates
#h = [2*i for i in range(26,150)] #when desperate
g = j + h

if 'df_loop_reg' in globals():
    print('this var exists')
    del df_loop_reg
    if 'df_loop_reg' in globals():
        print('delete failed')
    else:
        print('delete success')
else:
    print('not in globals')
    
if 'df_loop_ext' in globals():
    print('this var exists')
    del df_loop_ext
    if 'df_loop_ext' in globals():
        print('delete failed')
    else:
        print('delete success')
else:
    print('not in globals')

    
# these variables will be declared below!

lambda_gamma = 0.003

summary_path = r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\He_code_new\dim_num_mb_val_test\packed\5_rates" + '\\' + r"summary_recheck.xlsx"
good_path = r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\He_code_new\dim_num_mb_val_test\packed\5_rates\good_rates_recheck"
bad_path = r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\He_code_new\dim_num_mb_val_test\packed\5_rates\bad_rates_recheck"


not in globals
not in globals


The discovery occurs in the next cell.
Parameters:
lambda_gamma (prev cell): penalty on the size of the dimensionless number
lambda_beta: penalty on the number of basis functions used in the polynomial regression

n_dimensionless: # of dimensionless numbers to discover.
poly_order: polynomial order. Linear is order 1.
poly_mapping: for the dimensionless number, what power each one has in order. See below.

para_threshold: threshold for gamma. Below this threshold, the gamma is treated as if it were zero.
beta_threshold: threshold for beta. Belwo this threshold, the beta is treated as if it were zero.

In [27]:


for i in g:
    seed = i
    #find which hyperparameter performed best

    # idx = np.arange(lg_set.shape[0])
    # best_hp = idx[array_metric_num[:,0]==np.max(array_metric_num)]

    # if len(best_hp.shape)>1:
    #     print('the following are equivalent ',best_hp)
    #     print(best_hp.shape)
    #     best_hp = best_hp[0,1]


    #22 best so far, R2 = 0.998 for lambda = 0.001
    #14 best so far, R2 = 0.945
    ndimensionless = 1
    #lambda_gamma = 0.003 #lambda_gamma = 0.01, #replaced

    poly_order = 1 #was 2? but linear is a first order poly.

    poly_mapping = np.array([[0],
                            [1]])

    #poly_mapping = np.array([[0, 0],
    #                         [1, 0],
    #                         [0, 1],
    #                         [2, 0],
    #                         [1, 1],
    #                         [0, 2]])
    lambda_beta = 0.01 #maybe cut in half idk
    w_array = np.array(fff._basis_col)  #this is the w array, that is, the columns in Null(D)
    gamma_name = ['y'+str(id) for id in range(0,fff.basis_col.shape[1]) ]
    beta_name = ['b'+str(id) for id in range(0,poly_mapping.shape[0]) ]
    poly_name = ['dim'+str(id+1) for id in range(0,ndimensionless)]



    metric = 'r2'
    para_threshold = 0.01 #vs 0.1?
    beta_threshold = 0.005
    training_epochs =10000
    score = []

    import numpy as np
    from sklearn.model_selection import train_test_split
    
     #This first part of the cell, we fit without testing the performance on extrapolation. That comes later.
        
        

    #create test set
    X_train_val, X_test, y_train_val, y_test = train_test_split(fff.X, fff.y, test_size=0.15, random_state=42)

    # create validation and training set  ##unused!
    # X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)
    torch.manual_seed(seed)

    X_train_val, X_test, y_train_val, y_test = train_test_split(fff.X, fff.y, test_size=0.2, random_state=42)



    best_hp = lambda_gamma


    model_train = PiLinearRegressionViaTorch.TrainHolder(X_train_val, y_train_val, poly_mapping.shape[0], fff.y.shape[-1], poly_mapping, fff.basis_col, ndimensionless, lambda_gamma, lambda_beta, lowest_para_threshold=para_threshold)
    metric_num = model_train.train(training_epochs, True, val_x=X_test, val_y =y_test , metric=metric, norm_on='null_space')

    model_train.model.load_state_dict(torch.load('best_model.pt'))


    r2 = model_train.get_validation_metric(X_test,y_test,metric)
    r2_reg = r2
    print(r2)

    paras = [j for j in model_train.model.parameters()]
    gamma = paras[0].detach()
    beta = paras[1].detach()

    #clamp them
    t = gamma*(torch.abs(gamma) > para_threshold)
    beta_prune = beta*(torch.abs(beta) > beta_threshold)


    #store values from training
    best_metric_num = np.array(metric_num)
    best_beta = beta_prune

    best_dim1_gamma = np.asarray(t.transpose(0,1)[0]) # gamma corresponding to dimensionless number 1
    best_cardinality_dim1_gamma = np.sum(np.where(best_dim1_gamma==0,0,1)) #non-zero elements corresponding to gamma for dim # 1
    best_dim1_w = np.matmul(w_array, best_dim1_gamma)  # w corresponding to dimensionless number 1
    best_cardinality_dim1_w = np.sum(np.where(best_dim1_w==0,0,1)) #non-zero elements in w corresponding to dim #1

    print(t)
    print(np.asarray(t.transpose(0,1)[0]))
    print(best_dim1_gamma)



    if ndimensionless >=2:
        best_dim2_gamma = np.asarray(t.transpose(0,1)[1]) # gamma corresponding to dimensionless number 2
        best_cardinality_dim2_gamma = np.sum(np.where(best_dim2_gamma==0,0,1)) #non-zero elements corresponding to gamma for dim # 2
        best_dim2_w = np.matmul(w_array, best_dim2_gamma)  # w corresponding to dimensionless number 2
        best_cardinality_dim2_w = np.sum(np.where(best_dim2_w==0,0,1)) #non-zero elements in w corresponding to dim #2


    #r2, lambda, beta vector , cardinality 
    best_data_dim1 = np.hstack((best_metric_num.reshape(1,1), np.array([para_threshold]).reshape(1,1)  ,np.array([best_hp]).reshape(1,1),  best_beta[0].reshape(1,poly_mapping.shape[0]),  best_cardinality_dim1_gamma.reshape(1,1) ,best_dim1_gamma.reshape(1,best_dim1_gamma.shape[0]) ,      best_dim1_w.reshape(1,best_dim1_w.shape[0]),   best_cardinality_dim1_w.reshape(1,1)))
    data_name =  [ metric] + ['clamp'] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[gamma_name[i] for i in range(0,len(gamma_name))]+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1']

    df_best_dim1 = pd.DataFrame(data=best_data_dim1, columns = [ metric] + ['clamp'] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[gamma_name[i] for i in range(0,len(gamma_name))]+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1'])

    ##### BELOW MUST BE CORRECTED#####
    if ndimensionless >=2:
        
        best_data_dim2 = np.hstack((best_metric_num.reshape(1,1),   best_hp.reshape(1,1),  best_beta[0].reshape(1,poly_mapping.shape[0]),   best_cardinality_dim2_gamma.reshape(1,1),    best_dim2_w.reshape(1,best_dim1_w.shape[0]),   best_cardinality_dim2_gamma.reshape(1,1)))
        df_best_dim2 = pd.DataFrame(data=best_data_dim2, columns = [ metric] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1'])

    #df_best_dim1 = pd.DataFrame(data=D_in,columns=variables) #first sheet, list dimension vector
    #####
    model_train.model.load_state_dict(torch.load('best_model.pt'))
    print([i for i in model_train.model.parameters()])
    paras = [j for j in model_train.model.parameters()]
    gamma = paras[0].detach()
    beta = paras[1].detach()

    #clamp them
    t = gamma*(torch.abs(gamma) > para_threshold)
    beta_prune = beta*(torch.abs(beta) > beta_threshold)
    print(t)
    print(np.asarray(t.transpose(0,1)[0]))
    print(best_dim1_gamma)
    # if you need the orignial pis
    readable_para = torch.cat((t.transpose(0,1).reshape(-1), torch.tensor([-8, poly_order])))
    ori_pis = fff.get_symbolic_pis(readable_para, variables)
    ori_pis[0]
    #ori_pis[1]
    
    
    # store the data in data_frames
    df0 = pd.DataFrame(data=D_in,columns=variables) #first sheet, list dimension vector

    df1 = pd.DataFrame(data=np.transpose(w_array),columns=variables) #second sheet, list nullspace vectors and associated variables


    df4 =  pd.DataFrame(data = best_beta, columns =  [beta_name[i] for i in range(0,len(beta_name))] )
    df5 = pd.DataFrame(data = poly_mapping, columns = [poly_name[i] for i in range(0,ndimensionless)])
    

    
    #PREPARE TO DO THE EXTRAPOLATION TEST

    #PREPARE TO DO THE EXTRAPOLATION TEST

    #PREPARE TO DO THE EXTRAPOLATION TEST
    
    

    # delete old variables

    variable_names = ['X_test', 'Y_test', 'X_train', 'X_val', 'y_train', 'y_val', 'X_train_val', 'Y_train_val']
    for var_name in variable_names:
        if var_name in locals() or var_name in globals():
            target_dict = locals() if var_name in locals() else globals()
            del target_dict[var_name]
            #print(f"Deleted variable: {var_name}")
            
    
    # Refit, but we test extrapolation.

    
    X_test, Y_test, X_train_val, y_train_val = top_split_y(fff.X,fff.y,5)
    
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)
    # X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)

    
    print(X_test.shape)
    print(Y_test.shape)

    x_test = X_test
    y_test = Y_test
    print(y_test)
    print(y_train_val)
    training_epochs =10000


    best_hp = lambda_gamma


    model_train = PiLinearRegressionViaTorch.TrainHolder(X_train, y_train, poly_mapping.shape[0], fff.y.shape[-1], poly_mapping, fff.basis_col, ndimensionless, lambda_gamma, lambda_beta, lowest_para_threshold=para_threshold)
    metric_num = model_train.train(training_epochs, True, val_x=X_val, val_y =y_val , metric=metric, norm_on='null_space')

    model_train.model.load_state_dict(torch.load('best_model.pt'))


    r2 = model_train.get_validation_metric(X_val,y_val,metric)
    r2_ext = r2
    print(r2)
    
    #check its performance on the test set
    r2_ext_test = model_train.get_validation_metric(X_test,y_test,metric)
    print(r2_ext_test)

    paras = [j for j in model_train.model.parameters()]
    gamma = paras[0].detach()
    beta = paras[1].detach()

    #clamp them
    t = gamma*(torch.abs(gamma) > para_threshold)
    beta_prune = beta*(torch.abs(beta) > beta_threshold)


    #store values from training
    best_metric_num = np.array(metric_num)
    best_beta = beta_prune

    best_dim1_gamma = np.asarray(t.transpose(0,1)[0]) # gamma corresponding to dimensionless number 1
    best_cardinality_dim1_gamma = np.sum(np.where(best_dim1_gamma==0,0,1)) #non-zero elements corresponding to gamma for dim # 1
    best_dim1_w = np.matmul(w_array, best_dim1_gamma)  # w corresponding to dimensionless number 1
    best_cardinality_dim1_w = np.sum(np.where(best_dim1_w==0,0,1)) #non-zero elements in w corresponding to dim #1

    print(t)
    print(np.asarray(t.transpose(0,1)[0]))
    print(best_dim1_gamma)



    if ndimensionless >=2:
        best_dim2_gamma = np.asarray(t.transpose(0,1)[1]) # gamma corresponding to dimensionless number 2
        best_cardinality_dim2_gamma = np.sum(np.where(best_dim2_gamma==0,0,1)) #non-zero elements corresponding to gamma for dim # 2
        best_dim2_w = np.matmul(w_array, best_dim2_gamma)  # w corresponding to dimensionless number 2
        best_cardinality_dim2_w = np.sum(np.where(best_dim2_w==0,0,1)) #non-zero elements in w corresponding to dim #2


    #r2, lambda, beta vector , cardinality 
    best_data_dim1 = np.hstack(( r2_ext_test.reshape(1,1), best_metric_num.reshape(1,1), np.array([para_threshold]).reshape(1,1)  ,np.array([best_hp]).reshape(1,1),  best_beta[0].reshape(1,poly_mapping.shape[0]),  best_cardinality_dim1_gamma.reshape(1,1) ,best_dim1_gamma.reshape(1,best_dim1_gamma.shape[0]) ,      best_dim1_w.reshape(1,best_dim1_w.shape[0]),   best_cardinality_dim1_w.reshape(1,1)))
    data_name = ['r2_ext_test'] + [ metric] + ['clamp'] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[gamma_name[i] for i in range(0,len(gamma_name))]+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1']

    df_best_dim1_ext = pd.DataFrame(data=best_data_dim1, columns =['r2_ext_test'] + [ metric] + ['clamp'] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[gamma_name[i] for i in range(0,len(gamma_name))]+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1'])

    ##### BELOW MUST BE CORRECTED#####
    if ndimensionless >=2:
        
        best_data_dim2 = np.hstack((best_metric_num.reshape(1,1),   best_hp.reshape(1,1),  best_beta[0].reshape(1,poly_mapping.shape[0]),   best_cardinality_dim2_gamma.reshape(1,1),    best_dim2_w.reshape(1,best_dim1_w.shape[0]),   best_cardinality_dim2_gamma.reshape(1,1)))
        df_best_dim2 = pd.DataFrame(data=best_data_dim2, columns = [ metric] + ['lambda']+ [beta_name[i] for i in range(0,len(beta_name))]+['cardinality_of_gamma_dim1']+[variables[i] for i in range(0,len(variables))]+['cardinality_of_w_dim1'])

    #df_best_dim1 = pd.DataFrame(data=D_in,columns=variables) #first sheet, list dimension vector
    #####
    model_train.model.load_state_dict(torch.load('best_model.pt'))
    print([i for i in model_train.model.parameters()])
    paras = [j for j in model_train.model.parameters()]
    gamma = paras[0].detach()
    beta = paras[1].detach()

    #clamp them
    t = gamma*(torch.abs(gamma) > para_threshold)
    beta_prune = beta*(torch.abs(beta) > beta_threshold)
    print(t)
    print(np.asarray(t.transpose(0,1)[0]))
    print(best_dim1_gamma)
    # if you need the orignial pis
    readable_para = torch.cat((t.transpose(0,1).reshape(-1), torch.tensor([-8, poly_order])))
    ori_pis = fff.get_symbolic_pis(readable_para, variables)
    ori_pis[0]
    #ori_pis[1]

    ori_pis[0]
    print(t)
    print(np.asarray(t.transpose(0,1)[0]))
    print(best_dim1_gamma)
    # put save_path decision here
        #set save path--remove this chunk of code to the bottom?

    #current_path = r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs\dimensionless_numbers_mb\limited_terms\single_lambdas"
    if r2_reg >0.3 or r2_ext >0.3:
        current_path = good_path
    else:
        current_path = bad_path
    #new_folder =r'mb_1dim_v1_pt0001_pt_01_order1_golden_child'
    new_folder = r'clamp'+str(para_threshold)+r'_' + str(ndimensionless)+r'dim_order'+str(poly_order)+r'_lambda'+str(lambda_gamma)+r"_seed"+str(seed)
    new_path = current_path+'\\'+new_folder
    if os.path.exists(new_path):
        print(f"File '{new_path}' already exists.")
    else:
        os.mkdir(new_path)       
        print(f"File '{new_path}' did not exist.")

    file_path = new_path+'\\' +'_seed'+str(seed)+ '.xlsx'
    fig_path_dim1 = new_path+'\\'+new_folder+"dim1"
    fig_path_dim2 = new_path+'\\'+new_folder+"dim2"
    print(fig_path_dim1)
    # File path of the Excel file

    if os.path.exists(file_path):
        # Delete the file
        os.remove(file_path)
        print(f"File '{file_path}' deleted successfully.")
    else:
        print(f"File '{file_path}' does not exist.")



    with pd.ExcelWriter(file_path) as writer:  
        df0.to_excel(writer, sheet_name='dimension_matrix')
        df1.to_excel(writer, sheet_name='null_space_matrix')
        df4.to_excel(writer, sheet_name='beta')
        df5.to_excel(writer, sheet_name='polynomial')
        df_best_dim1.to_excel(writer, sheet_name='best_dim_1')
    
    # now to store the summary and performance of this 'reg dim #'
    if 'df_loop_reg' in globals():
        df_loop_reg = pd.concat([df_loop_reg, df_best_dim1 ], ignore_index = True)

    else:
        df_loop_reg = df_best_dim1.copy(deep=True)
    
            
    # Now store the data from the extrapolation attempt. 
    # select file path to save xlsx sheet
    # and overwrite the dataframes (df's) 
    # then store these new data frames in a separate excel file
    
    
    # File path of the Excel file
    file_path = new_path+'\\' + '_extrapolation' +'_seed'+str(seed)+ '.xlsx'
    fig_path_dim1 = new_path+'\\'+new_folder+'_extrapolation_' +"dim1"
    fig_path_dim2 = new_path+'\\'+new_folder+'_extrapolation_' +"dim2"
    print(fig_path_dim1)

    if os.path.exists(file_path):
        # Delete the file
        os.remove(file_path)
        print(f"File '{file_path}' deleted successfully.")
    else:
        print(f"File '{file_path}' does not exist.")


    df0_ext = pd.DataFrame(data=D_in,columns=variables) #first sheet, list dimension vector

    df1_ext = pd.DataFrame(data=np.transpose(w_array),columns=variables) #second sheet, list nullspace vectors and associated variables


    df4_ext =  pd.DataFrame(data = best_beta, columns =  [beta_name[i] for i in range(0,len(beta_name))] )
    df5_ext = pd.DataFrame(data = poly_mapping, columns = [poly_name[i] for i in range(0,ndimensionless)])
    with pd.ExcelWriter(file_path) as writer:  
        df0_ext.to_excel(writer, sheet_name='dimension_matrix')
        df1_ext.to_excel(writer, sheet_name='null_space_matrix')
        df4_ext.to_excel(writer, sheet_name='beta')
        df5_ext.to_excel(writer, sheet_name='polynomial')
        df_best_dim1_ext.to_excel(writer, sheet_name='best_dim_1')
    
    # now to store the summary of the 'ext dim #' for placement in an excel so I can compare them all!
    if 'df_loop_ext' in globals():
        df_loop_ext = pd.concat([df_loop_ext, df_best_dim1_ext ], ignore_index = True)
        
    else:
        df_loop_ext = df_best_dim1_ext.copy(deep=True)
    
        
    print(t)
    print(np.asarray(t.transpose(0,1)[0]))
    print(best_dim1_gamma)
#write df loops to good path!
with pd.ExcelWriter(summary_path) as writer:  
    df_loop_reg.to_excel(writer, sheet_name='reg dimension number')
    df_loop_ext.to_excel(writer, sheet_name='ext dimension number')

loss: 1.314030549771643 at iteration 9999, val_loss: -0.05525771521283929, best_val_loss: -2.8140152741329416e-07: 100%|██████████| 10000/10000 [00:45<00:00, 221.03it/s] 


-2.8140152741329416e-07
tensor([[-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [0.],
        [-0.],
        [0.]], dtype=torch.float64)
[-0. -0. -0. -0. -0.  0. -0.  0.]
[-0. -0. -0. -0. -0.  0. -0.  0.]
[Parameter containing:
tensor([[-0.0008],
        [-0.0015],
        [-0.0006],
        [-0.0062],
        [-0.0018],
        [ 0.0008],
        [-0.0009],
        [ 0.0009]], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([[ 1.3604e+00, -7.2856e-04]], dtype=torch.float64, requires_grad=True)]
tensor([[-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [0.],
        [-0.],
        [0.]], dtype=torch.float64)
[-0. -0. -0. -0. -0.  0. -0.  0.]
[-0. -0. -0. -0. -0.  0. -0.  0.]
flag 1
Y.shape  = (235, 1)
X.shape  = (235, 1)
test_size =  11
sorted indices =  [ 59  60  55 123 151  61  47 119  63  57  49]
flag 2
Y_test.shape  =  (11, 1)
X_test.shape  = (11, 11)
flag 3
Y_train.shape  = (224, 1)
X_train.shape  = (224, 1

loss: 0.3346913903787605 at iteration 9999, val_loss: 0.3657595144261657, best_val_loss: 0.3814933301820638: 100%|██████████| 10000/10000 [00:47<00:00, 211.74it/s] 


0.3814933301820638
-65.16314084735181
tensor([[-0.4936],
        [ 0.0353],
        [ 0.0000],
        [ 0.5377],
        [-0.1530],
        [ 0.4773],
        [-0.3670],
        [ 0.1820]], dtype=torch.float64)
[-0.49362616  0.03526169  0.          0.53767748 -0.15304898  0.4773034
 -0.36704109  0.18203403]
[-0.49362616  0.03526169  0.          0.53767748 -0.15304898  0.4773034
 -0.36704109  0.18203403]
[Parameter containing:
tensor([[-0.4936],
        [ 0.0353],
        [ 0.0024],
        [ 0.5377],
        [-0.1530],
        [ 0.4773],
        [-0.3670],
        [ 0.1820]], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([[0.2480, 0.4382]], dtype=torch.float64, requires_grad=True)]
tensor([[-0.4936],
        [ 0.0353],
        [ 0.0000],
        [ 0.5377],
        [-0.1530],
        [ 0.4773],
        [-0.3670],
        [ 0.1820]], dtype=torch.float64)
[-0.49362616  0.03526169  0.          0.53767748 -0.15304898  0.4773034
 -0.36704109  0.18203403]
[-0.4936261

loss: 0.560924857497425 at iteration 9999, val_loss: 0.5907277313066389, best_val_loss: 0.5972401046808544: 100%|██████████| 10000/10000 [00:52<00:00, 191.35it/s]  


...

0.35691599187838274
-56.50074555570564
tensor([[-0.6059],
        [ 0.0972],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [ 1.3010],
        [-0.3559],
        [-0.0000]], dtype=torch.float64)
[-0.60589423  0.09721168 -0.         -0.         -0.          1.30100933
 -0.35587051 -0.        ]
[-0.60589423  0.09721168 -0.         -0.         -0.          1.30100933
 -0.35587051 -0.        ]
[Parameter containing:
tensor([[-6.0589e-01],
        [ 9.7212e-02],
        [-1.6222e-04],
        [-3.6561e-03],
        [-7.9318e-03],
        [ 1.3010e+00],
        [-3.5587e-01],
        [-5.9269e-04]], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([[1.5717e-04, 2.6812e-01]], dtype=torch.float64, requires_grad=True)]
tensor([[-0.6059],
        [ 0.0972],
        [-0.0000],
        [-0.0000],
        [-0.0000],
        [ 1.3010],
        [-0.3559],
        [-0.0000]], dtype=torch.float64)
[-0.60589423  0.09721168 -0.         -0.         -0.          1.30

loss: 0.4019092078067161 at iteration 9999, val_loss: 0.5982732720329027, best_val_loss: 0.6060162540176102: 100%|██████████| 10000/10000 [00:43<00:00, 230.08it/s]


0.6060162540176102
tensor([[-0.6321],
        [ 1.5431],
        [ 0.1031],
        [-0.0000],
        [ 0.0000],
        [ 1.2458],
        [-0.5591],
        [-1.3124]], dtype=torch.float64)
[-0.63209538  1.54309112  0.10305317 -0.          0.          1.2457669
 -0.55913805 -1.31235742]
[-0.63209538  1.54309112  0.10305317 -0.          0.          1.2457669
 -0.55913805 -1.31235742]
[Parameter containing:
tensor([[-6.3210e-01],
        [ 1.5431e+00],
        [ 1.0305e-01],
        [-4.7169e-03],
        [ 9.4016e-06],
        [ 1.2458e+00],
        [-5.5914e-01],
        [-1.3124e+00]], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([[2.5511e-05, 1.0795e-01]], dtype=torch.float64, requires_grad=True)]
tensor([[-0.6321],
        [ 1.5431],
        [ 0.1031],
        [-0.0000],
        [ 0.0000],
        [ 1.2458],
        [-0.5591],
        [-1.3124]], dtype=torch.float64)
[-0.63209538  1.54309112  0.10305317 -0.          0.          1.2457669
 -0.55913805 -1.

loss: 0.19333294901602027 at iteration 9999, val_loss: 0.7418238564821564, best_val_loss: 0.7418167950494992: 100%|██████████| 10000/10000 [00:52<00:00, 189.52it/s] 


0.7418238564821564
-29.604150132269826
tensor([[-6.5219],
        [ 5.8699],
        [ 5.7939],
        [ 6.4271],
        [-0.0000],
        [ 0.8241],
        [-0.1325],
        [-0.0000]], dtype=torch.float64)
[-6.52187108  5.86992151  5.79393123  6.4270538  -0.          0.82409752
 -0.13251551 -0.        ]
[-6.52187108  5.86992151  5.79393123  6.4270538  -0.          0.82409752
 -0.13251551 -0.        ]
[Parameter containing:
tensor([[-6.5219e+00],
        [ 5.8699e+00],
        [ 5.7939e+00],
        [ 6.4271e+00],
        [-5.2487e-04],
        [ 8.2410e-01],
        [-1.3252e-01],
        [-1.1332e-03]], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([[0.0019, 0.0704]], dtype=torch.float64, requires_grad=True)]
tensor([[-6.5219],
        [ 5.8699],
        [ 5.7939],
        [ 6.4271],
        [-0.0000],
        [ 0.8241],
        [-0.1325],
        [-0.0000]], dtype=torch.float64)
[-6.52187108  5.86992151  5.79393123  6.4270538  -0.          0.82409752
 

In [28]:
with pd.ExcelWriter(summary_path) as writer:  
    df_loop_reg.to_excel(writer, sheet_name='reg dimension number')
    df_loop_ext.to_excel(writer, sheet_name='ext dimension number')

In [29]:
import pygame
import time
import numpy as np

# Initialize pygame mixer
pygame.mixer.init()

# Define the sample rate and duration of each tone
sample_rate = 44100  # Adjust as needed
duration = 0.5  # Adjust as needed

# Define notes and their corresponding frequencies
notes = {
    'C4': 261.63,
    'D4': 293.66,
    'E4': 329.63,
    'F4': 349.23,
    'G4': 392.00,
    'A4': 440.00,
    'B4': 493.88,
    'C5': 523.25
}

# Define the melody
melody = ['E4', 'D4', 'C4', 'D4', 'E4', 'E4', 'E4', 'D4', 'D4', 'D4', 'E4', 'G4', 'G4', 'E4', 'D4', 'C4', 'D4', 'E4', 'E4', 'E4', 'E4', 'D4', 'D4', 'E4', 'D4', 'C4']

# Play the melody
for note in melody:
    frequency = notes[note]
    waveform = pygame.sndarray.make_sound(pygame.sndarray.array([[int(4096 * 0.5 * np.sin(2.0 * np.pi * frequency * x / sample_rate)) for _ in range(2)] for x in range(int(sample_rate * duration))]))
    waveform.play()
    time.sleep(duration)


ModuleNotFoundError: No module named 'pygame'

In [ ]:
import pygame
import time

# Initialize pygame mixer
pygame.mixer.init()

# Define notes and their corresponding frequencies
notes = {
    'D4': 293.66,
    'A4': 440.00,
    'B4': 493.88,
    'F#4': 369.99,
    'G4': 392.00,
    'E4': 329.63
}

# Define the melody for the first few bars of Pachelbel's Canon
melody = ['D4', 'A4', 'B4', 'F#4', 'G4', 'D4', 'G4', 'A4', 'D4', 'B4', 'G4', 'D4', 'F#4', 'G4', 'A4', 'B4', 'A4', 'G4', 'F#4', 'G4', 'F#4', 'E4', 'A4', 'D4', 'D4', 'B4', 'G4', 'D4', 'A4', 'A4', 'B4', 'F#4', 'G4', 'D4', 'G4', 'A4', 'B4', 'A4', 'G4', 'F#4', 'G4', 'A4', 'B4', 'A4', 'G4', 'F#4', 'G4', 'F#4', 'E4']

# Play the melody
for note in melody:
    frequency = notes[note]
    waveform = pygame.sndarray.make_sound(pygame.sndarray.array([[int(4096 * 0.5 * np.sin(2.0 * np.pi * frequency * x / sample_rate)) for _ in range(2)] for x in range(int(sample_rate * duration))]))
    waveform.play()
    time.sleep(0.3)  # Adjust the duration of each note as needed


In [ ]:
X_test, Y_test, X_train_val, y_train_val = top_split_y(fff.X,fff.y,5)
print(X_test.shape)
print(Y_test.shape)
print(X_train_val.shape)
print(y_train_val.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

# This code is to check the D matrix used in the paper (6 rates, packed) and the one used above are the same.

In [25]:
D_in = np.array(
[
    [ 0 , 3, -3,  3,  1,  0,  2, -3,  2, -3,  1, -3,  3],
    [ 1, -1,  0,  0, -1,  1,  0,  0,  0,  0, -1,  0, -1],
    [ 0, -1,  1,  0,  0,  0,  0,  1,  0,  1,  0,  1, -1]
]
)


# D_in: find W for paper
D_in_paper = np.array(
[
    [ 1,  1,  2,  3,  0,  0,  3,  3, -3, -3, -3, -3,  2],
    [-1, -1,  0,  0,  1,  1, -1, -1,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0, -1, -1,  1,  1,  1,  1,  0]
]
)

print(D_in)
print(D_in_paper)

[[ 0  3 -3  3  1  0  2 -3  2 -3  1 -3  3]
 [ 1 -1  0  0 -1  1  0  0  0  0 -1  0 -1]
 [ 0 -1  1  0  0  0  0  1  0  1  0  1 -1]]
[[ 1  1  2  3  0  0  3  3 -3 -3 -3 -3  2]
 [-1 -1  0  0  1  1 -1 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1 -1  1  1  1  1  0]]


In [26]:
df_out = pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]
n_e = np.array(df_out.iloc[:,3])
df_out = pd.read_excel(df_2, sheet_name='other_data').iloc[:,1:]
n_g = np.array(df_out.iloc[:,4])
n_e_no_dim = n_e/n_g
#print(n_e)
#print(n_e_no_dim)
output = rescale_vec(n_e_no_dim)
#print(output)
D_out = np.array(
    [
        [0.],
        [0.],
        [0.],
    ],

)


fff = DimensionlessLearning(D_in, D_out)
with np.printoptions(precision = 3):
    print(fff.basis_col.numpy())
    print(fff.basis_col.numpy().shape)

[[ 1.     1.    -1.     0.     1.     0.     1.     1.     1.     0.   ]
 [ 1.     0.     0.     0.     1.     0.     1.     0.     1.    -1.   ]
 [ 1.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.    -0.333  0.    -0.667  0.    -0.667  0.    -0.333  0.     0.   ]
 [ 0.     1.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     1.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     1.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     1.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     1.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     1.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     1.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     1.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     1.   ]]
(13, 10)


In [93]:
mat = fff.basis_col.numpy()
print(np.matmul(D_in_paper,mat))
print(np.matmul(D_in,mat))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 3.   7.5  2.5  1.5 -5.   2.5 -2.5  1.5 -2.5  6. ]
 [-2.   0.   0.   2.   0.  -1.  -1.  -2.  -1.  -1. ]
 [-1.  -1.5 -0.5 -0.5  1.   0.5  1.5  0.5  1.5 -2. ]]


In [98]:
r = n_e/n_g
print(max(r))
print(min(r))

5.270352083652026e-13
4.4805095763361205e-15


In [94]:
def have_same_columns(a, b):
    # Set of column-tuples from each array
    set_a = {tuple(col) for col in a.T}
    set_b = {tuple(col) for col in b.T}
    return set_a == set_b
result = have_same_columns(D_in, D_in_paper)
print(result)

True


In [95]:
import numpy as np

# Example arrays
array1 = np.array([[1, 2, 3], [4, 5, 6]])
array2 = np.array([[3, 1, 2], [6, 4, 5]])

# Function to check if two arrays have the same columns regardless of order
def have_same_columns(a, b):
    # Set of column-tuples from each array
    set_a = {tuple(col) for col in a.T}
    set_b = {tuple(col) for col in b.T}
    return set_a == set_b

# Check if the arrays have the same columns
result = have_same_columns(array1, array2)
print(result)


True


In [61]:
D_in = np.delete(D_in, 0, axis = 1)
D_in_paper = np.delete(D_in_paper, 4, axis = 1)
print(D_in)
print(D_in_paper)

[[ 3 -3  3  1  0  2 -3  2 -3  1 -3  3]
 [-1  0  0 -1  1  0  0  0  0 -1  0 -1]
 [-1  1  0  0  0  0  1  0  1  0  1 -1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]]
[[ 1  1  2  3  0 -3 -3 -3 -3 -3 -3  2]
 [-1 -1  0  0  1 -1 -1  0  0  0  0  0]
 [ 0  0  0  0  0 -1 -1  1  1  1  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]]
